In [ ]:
"""
Installing the dependencies.
"""

!pip install pandas
!pip install scikit-learn
!pip install opencv-python
!pip install numpy
!pip install tensorflow
!pip install ultralytics

In [7]:

"""
Read the dataset and transform the labels to 0 or 1.
"""
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("image_labels.csv")
df['label'] = LabelEncoder().fit_transform(df['label'])
df["label"] = df["label"].replace({0: 1, 1: 0})

In [9]:
"""
Split the dataset into train, validation and test sets.
"""

X_train, X_test, y_train, y_test = train_test_split(df['filename'], df['label'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [10]:
"""
Read, resize and load images.
"""

import cv2
import numpy as np

def load_images(image_paths, img_size=(224, 224)):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, img_size)
        images.append(img)
    return np.array(images)


X_train = load_images(X_train)
X_val = load_images(X_val)
X_test = load_images(X_test)

In [23]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:100]:  
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Aumentei o dropout para regularização
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=out)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)
loss, acc = model.evaluate(X_test, y_test)

print(f'Test accuracy: {acc:.2f}')

45/45 [==============================] - 8s 176ms/step - loss: 1.5187 - accuracy: 0.9250
Test accuracy: 0.93


In [15]:
"""
Use the model to detect dangerous objects in a video.
"""

def detect_objects_in_video(video_path, model, threshold=0.5):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        img = cv2.resize(frame, (224, 224)) / 255.0
        pred = model.predict(np.expand_dims(img, axis=0))[0][0]
        
        label = 'DANGEROUS' if pred > threshold else 'SAFE'
        color = (0, 0, 255) if pred > threshold else (0, 255, 0)
        cv2.putText(frame, label, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow('Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
detect_objects_in_video("video.mp4", model)

In [24]:
"""
Method to show the gradient CAM (Class Activation Map) for the model.
Shows where the model paid attention to in the image to make the prediction.
"""

import tensorflow as tf
from tensorflow.keras.models import Model

def grad_cam(model, image, layer_name):
    img_array = np.expand_dims(image, axis=0) / 255.0
    grad_model = Model(inputs=model.input, outputs=[model.get_layer(layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]  # Como temos saída binária, usamos a primeira (facas)

    grads = tape.gradient(loss, conv_outputs)[0]  # Gradientes da camada
    weights = np.mean(grads, axis=(0, 1))

    cam = np.dot(conv_outputs[0], weights)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    cam = cv2.resize(cam, (image.shape[1], image.shape[0]))

    heatmap = np.uint8(255 * cam)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    final_image = cv2.addWeighted(image, 0.5, heatmap, 0.5, 0)

    return final_image


In [ ]:
image_path = "basket.png"
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224))

layer_name = "Conv_1"
result = grad_cam(model, image, layer_name)

cv2.imshow("Grad-CAM", result)
cv2.waitKey(0)
cv2.destroyAllWindows()